# Linkres cleanup and filtering

### Boilerplate

In [1]:
%load_ext lab_black
# python internal
import collections
import copy
import gc
from glob import glob
import h5py
import itertools
import os
import random
import re
import socket
import shutil
import subprocess
import sys

# conda/pip
import dask
import graphviz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
from tqdm import tqdm

# special packages on the DIGS
import py3Dmol
import pymol
import pyrosetta

# notebook magic
%matplotlib inline
%load_ext autoreload
%autoreload 2

print(os.getcwd())
print(socket.gethostname())

/mnt/home/pleung/projects/bistable_bundle/r4/hinges
dig204


### Generate state X and state Y with and without disulfides by doing LinkRes and fixbb design
I will use the serialization build of PyRosetta to enable recording user defined info about the designs.  
This enables downstream inline filtering and data analysis, as well as clustering by lineage.

Record rich information about residues that might be interesting to show/direct switching, GLY, HIS and TRP.  
Record info about sequences and save pdbs for X and Y, with and without disulfide staples

In [2]:
from pyrosetta.distributed.packed_pose.core import PackedPose


def linkres_msd(packed_pose_in=None, **kwargs) -> PackedPose:
    """
    Load pose. Design with linkres x3: state_X_nocys, state_Y_nocys,
    state_XY_nocys. Dump X and Y poses with and without disulfides (4 total).
    Return XY without the disulfides.
    """
    import os
    import pyrosetta
    import pyrosetta.distributed.io as io
    from pyrosetta.distributed import cluster
    from pyrosetta.rosetta.protocols.denovo_design.movers import FastDesign
    from pyrosetta.rosetta.protocols.rosetta_scripts import XmlObjects
    from pyrosetta.rosetta.protocols.task_operations import LinkResidues
    from pyrosetta.rosetta.core.pack.task.operation import (
        IncludeCurrent,
        InitializeFromCommandline,
        ExtraRotamersGeneric,
    )
    from pyrosetta.rosetta.protocols.task_operations import LimitAromaChi2Operation

    def parse_scoreinfo(pdb_path) -> dict:
        """
        Hacky unsafe way to load scoreinfo written to pdb by rosetta.
        Probably should add additional pdb strings to skip
        """
        with open(pdb_path, "r") as f:
            scores = {}
            pdb_strings = ["HEADER", "EXPDTA", "SSBOND", "ATOM", "TER", "REMARK", "#"]
            for line in f:
                if line == "\n":  # skip empty lines
                    pass
                # skip pdb specific rows
                elif any(pdb_string in line for pdb_string in pdb_strings):
                    pass
                else:
                    key_value = line.rstrip().split()
                    # handle pose energies table
                    try:
                        assert len(key_value) == 2
                    except AssertionError:  # pose energies have many values per (residue) key
                        continue
                    key, value = tuple(key_value)
                    if any(c.isalpha() for c in value):  # handle as str below
                        pass
                    elif "," in value:  # handle as str below
                        pass
                    elif "." in value:  # if it has a . it maybe a float
                        try:
                            value = float(value)
                        except ValueError:
                            pass
                    else:  # if all else fails maybe an int
                        try:
                            value = int(value)
                        except ValueError:
                            pass
                    scores[key] = value  # if all typecasting failed it is still a str
        return scores

    def score(packed_pose_in: PackedPose, **kwargs) -> PackedPose:
        """
        Sequence
        Compute HIS SASA for X, Y, (compute delta on df)
        Compute buried HIS (probably in interface)
        Count GLY near interface, (compute glycine count on df)
        Compute TRP SASA for X, Y, (compute delta on df)
        Count TRP nearby aromatic non-TRP neighbors
        Assumes middle is where the switch is split originally, enabling forward and backward SSElement selection approach
        @pleung
        """
        import pyrosetta
        import pyrosetta.distributed.io as io
        from pyrosetta.distributed.tasks.rosetta_scripts import (
            SingleoutputRosettaScriptsTask,
        )

        scores = kwargs["score_dict"]
        xml = """
        <ROSETTASCRIPTS>
            <SCOREFXNS>
            </SCOREFXNS>
            <RESIDUE_SELECTORS>
                <ResidueName name="GLY" residue_name3="GLY" />
                <ResidueName name="HIS" residue_name3="HIS" />
                <ResidueName name="TRP" residue_name3="TRP" />
                <ResidueName name="aro" residue_name3="TYR,PHE,HIS" />
                <Layer name="surface" select_core="false" select_boundary="false" select_surface="true"
                    use_sidechain_neighbors="true"/>
                <Layer name="boundary" select_core="false" select_boundary="true" select_surface="false" 
                    use_sidechain_neighbors="true"/>
                <Layer name="core" select_core="true" select_boundary="false" select_surface="false" 
                    use_sidechain_neighbors="true"/>
                <SecondaryStructure name="sheet" overlap="0" minH="3" minE="2" include_terminal_loops="false" 
                    use_dssp="true" ss="E"/>
                <SecondaryStructure name="entire_loop" overlap="0" minH="3" minE="2" include_terminal_loops="true" 
                    use_dssp="true" ss="L"/>
                <SecondaryStructure name="entire_helix" overlap="0" minH="3" minE="2" include_terminal_loops="false"
                    use_dssp="true" ss="H"/>
                <And name="helix_cap" selectors="entire_loop">
                    <PrimarySequenceNeighborhood lower="1" upper="0" selector="entire_helix"/>
                </And>
                <And name="helix_start" selectors="entire_helix">
                    <PrimarySequenceNeighborhood lower="0" upper="1" selector="helix_cap"/>
                </And>
                <And name="helix" selectors="entire_helix">
                    <Not selector="helix_start"/>
                </And>
                <And name="loop" selectors="entire_loop">
                    <Not selector="helix_cap"/>
                </And>
                <SSElement name="near_int" selection="{pre},H,S" to_selection="-{post},H,E" chain="A" reassign_short_terminal_loop="2" />            
                <Neighborhood name="near_TRP" selector="TRP" distance="6.0" />
                <And name="GLY_near_int" selectors="near_int,HIS" />
                <And name="core_HIS" selectors="core,HIS" />
                <And name="aro_near_TRP" selectors="aro,near_TRP" />
            </RESIDUE_SELECTORS>
            <TASKOPERATIONS>
            </TASKOPERATIONS>
            <FILTERS>
                <BuriedSurfaceArea name="HIS_burial" residue_selector="HIS" confidence="0" />
                <BuriedSurfaceArea name="TRP_burial" residue_selector="TRP" confidence="0" />
                <ResidueCount name="buried_HIS" residue_selector="core_HIS" />
                <ResidueCount name="near_int_GLY" residue_selector="GLY_near_int" />
                <ResidueCount name="near_TRP_aro" residue_selector="aro_near_TRP" />
            </FILTERS>
            <MOVERS>
            </MOVERS>
            <PROTOCOLS>
                <Add filter="HIS_burial" />
                <Add filter="TRP_burial" />
                <Add filter="buried_HIS" />
                <Add filter="near_int_GLY" />
                <Add filter="near_TRP_aro" />
            </PROTOCOLS>
        </ROSETTASCRIPTS>
        """.format(
            pre=int(int(scores["pre_break_helix"]) - 1),
            post=int(int(scores["pre_break_helix"]) - 1),
        )
        # relax and score pose
        score = SingleoutputRosettaScriptsTask(xml)
        scored_ppose = score(packed_pose_in.pose.clone())
        update = scored_ppose.pose.scores
        pose = io.to_pose(scored_ppose.pose.clone())
        # update scores for values that were updated
        for key, value in scores.items():
            if key in update.keys():
                pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, key, update[key])
            else:
                pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, key, value)
        final_ppose = io.to_packed(pose)
        return final_ppose

    if packed_pose_in == None:
        pdb_path = kwargs["-s"]
    else:
        raise RuntimeError(
            "This method requires a pdb path, it cannot accept a PackedPose as input"
        )
    scores = parse_scoreinfo(pdb_path)
    ppose = io.pose_from_file(pdb_path)
    pose = io.to_pose(ppose)
    if kwargs["out_path"] == None:
        out_path = os.getcwd()
    else:
        out_path = kwargs["out_path"]
        os.makedirs(os.path.join(os.getcwd(), out_path), exist_ok=True)
    cys_X = scores["disulfide_at_X"].split(",")
    cys_Y = scores["disulfide_at_Y"].split(",")
    unique_cys = set(cys_X + cys_Y)
    # check whether disulfides are orthogonal
    design_resis = [int(x) for x in unique_cys]
    try:
        assert len(unique_cys) in [
            3,
            4,
        ]  # if there are less than 3 CYS or more than 4 we really f'd up at some point
    except AssertionError:
        print("There should be at least 3 unique CYS, but no more than 4")
        return None
    cys1_X, cys2_X = int(cys_X[0]), int(cys_X[1])
    cys1_Y, cys2_Y = int(cys_Y[0]), int(cys_Y[1])
    length = scores["total_length"]
    resis_matrix = [[x, int(x + length)] for x in design_resis]
    target_resis = ",".join(
        [str(x) for row in resis_matrix for x in row]
    )  # flatten matrix
    xml_string = """
    <ROSETTASCRIPTS>
        <SCOREFXNS>
            <ScoreFunction name="sfxn_design" weights="beta_nov16" >
                <Set use_hb_env_dep="true" />
                <Reweight scoretype="approximate_buried_unsat_penalty" weight="17" />
                <Set approximate_buried_unsat_penalty_burial_atomic_depth="3.5" />
                <Set approximate_buried_unsat_penalty_hbond_energy_threshold="-1.0" />
                <Set approximate_buried_unsat_penalty_natural_corrections1="true" />
                <Set approximate_buried_unsat_penalty_hbond_bonus_cross_chain="-7" />
                <Set approximate_buried_unsat_penalty_hbond_bonus_ser_to_helix_bb="1"/>                    
            </ScoreFunction>
        </SCOREFXNS>
        <RESIDUE_SELECTORS>
            <Index name="designable" resnums="{target_resis}" />
            <Neighborhood name="packable" selector="designable" distance="6.0" include_focus_in_subset="true" />
            <Not name="not_designable" selector="designable" />
            <Not name="not_packable" selector="packable" />
        </RESIDUE_SELECTORS>
        <TASKOPERATIONS>
            <OperateOnResidueSubset name="design" selector="designable"> # no CYS; GLY; PRO; TRP
                <RestrictAbsentCanonicalAASRLT aas="ADEFHIKLMNQRSTVY" />
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="pack" selector="not_designable">
                <RestrictToRepackingRLT/>
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="lock" selector="not_packable">
                <PreventRepackingRLT/>
            </OperateOnResidueSubset>
        </TASKOPERATIONS>
        <MOVERS>
            <SwitchChainOrder name="delete_Y" chain_order="1"/>
            <SwitchChainOrder name="delete_X" chain_order="2"/>
            <MutateResidue name="cys1_X" target="{cys1_X}" new_res="CYS" />
            <MutateResidue name="cys2_X" target="{cys2_X}" new_res="CYS" />
            <MutateResidue name="cys1_Y" target="{cys1_Y}" new_res="CYS" />
            <MutateResidue name="cys2_Y" target="{cys2_Y}" new_res="CYS" />
            <ForceDisulfides name="restore_X" 
                scorefxn="sfxn_design"
                disulfides="{cys1_X}:{cys2_X}"
                remove_existing="false"
                repack="true" />
            <ForceDisulfides name="restore_Y" 
                scorefxn="sfxn_design"
                disulfides="{cys1_Y}:{cys2_Y}"
                remove_existing="false"
                repack="true" />
            <FastRelax name="relax" 
                scorefxn="sfxn_design"
                repeats="2" 
                relaxscript="MonomerRelax2019"
                />
        </MOVERS>
        <FILTERS>
        </FILTERS>
        <SIMPLE_METRICS>
        </SIMPLE_METRICS>
        <APPLY_TO_POSE>
        </APPLY_TO_POSE>
        <PROTOCOLS>
        </PROTOCOLS>
    </ROSETTASCRIPTS>
    """.format(
        target_resis=target_resis,
        cys1_X=cys1_X,
        cys2_X=cys2_X,
        cys1_Y=cys1_Y,
        cys2_Y=cys2_Y,
    )
    # link all designable residues
    linkres = LinkResidues()
    for row in resis_matrix:
        linkres.add_group(",".join([str(x) for x in row]))
    # set taskops by extracting them from the xml_string
    xml_obj = XmlObjects.create_from_string(xml_string)
    design_task = xml_obj.get_task_operation("design")
    pack_task = xml_obj.get_task_operation("pack")
    lock_task = xml_obj.get_task_operation("lock")
    # set taskops
    task_factory = pyrosetta.rosetta.core.pack.task.TaskFactory()
    task_factory.push_back(IncludeCurrent())
    task_factory.push_back(InitializeFromCommandline())
    arochi = LimitAromaChi2Operation()
    arochi.include_trp(True)
    task_factory.push_back(arochi)
    ex1_ex2 = ExtraRotamersGeneric()
    ex1_ex2.ex1(True)
    ex1_ex2.ex2(True)
    task_factory.push_back(ex1_ex2)
    task_factory.push_back(design_task)
    task_factory.push_back(pack_task)
    task_factory.push_back(lock_task)
    task_factory.push_back(linkres)
    # set movemap
    mm = pyrosetta.rosetta.core.kinematics.MoveMap()
    mm.set_bb(False)
    mm.set_chi(True)
    mm.set_jump(False)
    sfxn_design = xml_obj.get_score_function("sfxn_design")
    fast_design = FastDesign(scorefxn_in=sfxn_design, standard_repeats=1)
    fast_design.cartesian(False)
    fast_design.set_task_factory(task_factory)
    fast_design.set_movemap(mm)
    fast_design.minimize_bond_angles(False)
    fast_design.minimize_bond_lengths(False)
    fast_design.min_type("lbfgs_armijo_nonmonotone")
    fast_design.ramp_down_constraints(False)

    name = pose.pdb_info().name()
    basename_no_ext = name.split("/")[-1].replace(".pdb", "", 1)
    if out_path[-1] != "/":
        out_path += "/"
    else:
        pass
    pose = pose.clone()
    fast_design.apply(pose)

    ref_pose = pose.clone()
    # get state X
    delete_Y = xml_obj.get_mover("delete_Y")
    delete_Y.apply(pose)
    ref_X = pose.clone()
    cys1 = xml_obj.get_mover("cys1_X")
    cys2 = xml_obj.get_mover("cys2_X")
    restore = xml_obj.get_mover("restore_X")
    relax = xml_obj.get_mover("relax")
    cys1.apply(pose)
    cys2.apply(pose)
    restore.apply(pose)
    # dump X
    pose.dump_pdb(os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_X.pdb"))
    scores["X_seq"] = pose.sequence()
    # get rmsd
    relax_X = ref_X.clone()
    relax.apply(relax_X)
    # dump X_nocys
    relax_X.dump_pdb(
        os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_X_nocys.pdb")
    )
    rmsd_calc = pyrosetta.rosetta.core.simple_metrics.metrics.RMSDMetric()
    rmsd_calc.set_rmsd_type(pyrosetta.rosetta.core.scoring.rmsd_atoms(3))
    rmsd_calc.set_run_superimpose(True)
    rmsd_calc.set_comparison_pose(ref_X)
    rmsd = rmsd_calc.calculate(relax_X)
    scores["rmsd_final_X"] = rmsd
    # get state Y
    pose = ref_pose.clone()
    delete_X = xml_obj.get_mover("delete_X")
    delete_X.apply(pose)
    ref_Y = pose.clone()
    cys1 = xml_obj.get_mover("cys1_Y")
    cys2 = xml_obj.get_mover("cys2_Y")
    restore = xml_obj.get_mover("restore_Y")
    relax = xml_obj.get_mover("relax")
    cys1.apply(pose)
    cys2.apply(pose)
    restore.apply(pose)
    # dump Y
    pose.dump_pdb(os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_Y.pdb"))
    scores["Y_seq"] = pose.sequence()
    # get rmsd
    relax_Y = ref_Y.clone()
    relax.apply(relax_Y)
    # dump Y_nocys
    relax_Y.dump_pdb(
        os.path.join(os.getcwd(), out_path + f"{basename_no_ext}_Y_nocys.pdb")
    )
    rmsd_calc = pyrosetta.rosetta.core.simple_metrics.metrics.RMSDMetric()
    rmsd_calc.set_rmsd_type(pyrosetta.rosetta.core.scoring.rmsd_atoms(3))
    rmsd_calc.set_run_superimpose(True)
    rmsd_calc.set_comparison_pose(ref_Y)
    rmsd = rmsd_calc.calculate(relax_Y)
    scores["rmsd_final_Y"] = rmsd
    assert relax_X.sequence() == relax_Y.sequence()
    scores["final_sequence"] = relax_X.sequence()
    # put in paths to X, Y, X_nocys, Y_nocys
    for suffix in ["X", "Y", "X_nocys", "Y_nocys"]:
        scores[f"{suffix}_path"] = os.path.join(
            os.getcwd(), out_path + f"{basename_no_ext}_{suffix}.pdb"
        )
    # score relax_X and relax_Y, update scores with those scores
    scored_X = score(io.to_packed(relax_X), score_dict=scores)
    scored_Y = score(io.to_packed(relax_Y), score_dict=scores)
    updates = []
    keys = ["HIS_burial", "TRP_burial", "buried_HIS", "near_int_GLY", "near_TRP_aro"]

    for chain, score_dict in [
        ("X", dict(scored_X.pose.scores)),
        ("Y", dict(scored_Y.pose.scores)),
    ]:
        updates.append(
            {
                f"{key}_{chain}": value
                for key, value in score_dict.items()
                if key in keys
            }
        )
    scores.update({**updates[0], **updates[1]})
    for key, value in scores.items():
        pyrosetta.rosetta.core.pose.setPoseExtraScore(ref_pose, key, value)
    ppose = io.to_packed(ref_pose)
    return ppose

# Setup Dask
Trying a adaptive SLURMCluster. to see the dashboard, forward port `8787` to `8000`:  
`local$ ssh -L 8000:localhost:8787 $USER@$HOSTNAME`  
now, the web UI is visible at `localhost:8000`  
if you"re using a local cluster make sure the node this notebook is on has the same 
number of workers as cores

TODO, force stdout to go into logs
TODO `extra=["--lifetime", "170m", "--lifetime-stagger", "4m"]`

In [ ]:
!echo $HOSTNAME
!echo $USER

In [5]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    cores=1,
    processes=1,
    job_cpu=1,
    local_directory="$TMPDIR/dask",
    log_directory="/mnt/home/pleung/logs/slurm_logs",
    memory="8GB",
    queue="medium",
    walltime="23:58:00",
    death_timeout=600,
)
print(cluster.job_script())
# scale between 0 and 1000 workers as needed
cluster.adapt(minimum=0, maximum=1020, wait_count=999) 
client = Client(cluster)
client

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /mnt/home/pleung/logs/slurm_logs/dask-worker-%J.err
#SBATCH -o /mnt/home/pleung/logs/slurm_logs/dask-worker-%J.out
#SBATCH -p medium
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=8G
#SBATCH -t 23:58:00

/home/pleung/.conda/envs/cereal/bin/python -m distributed.cli.dask_worker tcp://172.16.131.240:38091 --nthreads 1 --memory-limit 8.00GB --name name --nanny --death-timeout 600 --local-directory $TMPDIR/dask



Client Scheduler: tcp://172.16.131.240:38091 Dashboard: http://172.16.131.240:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
# client.close(); cluster.close()

# Set command line options, make tasks and submit to client
Try `tj` to start

In [ ]:
import logging
from pyrosetta.distributed.cluster.core import PyRosettaCluster
logging.basicConfig(level=logging.INFO)

pdbs = glob(os.path.join(os.getcwd(), "05_selected/tj/good_trp/*.pdb"))

options = { 
    "-out:level": "300",
    "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
    "-indexed_structure_store:fragment_store": "/net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5",
    "-quasisymmetry": "true",
    "-detect_disulf": "false",
    "-dunbrack_prob_buried": "0.8",
    "-dunbrack_prob_nonburied": "0.8", 
    "-dunbrack_prob_buried_semi": "0.8", 
    "-dunbrack_prob_nonburied_semi": "0.8",
}


# for i in range(5000):
#     print(int(i / 1000))

def create_tasks(pdbs, options):
    for pdb_path in pdbs:
        tasks = {"options": "-corrections::beta_nov16 true"}
        tasks["extra_options"] = options
        tasks["-s"] = pdb_path
        tasks["out_path"] = "06_states"
        tasks["set_logging_handler"] = "interactive"
        yield tasks
        
if not os.getenv("DEBUG"):
    output_path = os.path.join(os.getcwd(), "06_cleanup")
    PyRosettaCluster(
        tasks=create_tasks(pdbs, options),
        client=client,
        scratch_dir=output_path,
        output_path=output_path,
    ).distribute(protocols=[linkres_msd])

`conda env export --prefix /home/pleung/.conda/envs/cereal > environment.yml`
to reproduce this simulation later.
INFO:pyrosetta.distributed:maybe_init performing pyrosetta initialization: {'options': '-run:constant_seed 1 -multithreading:total_threads 1', 'extra_options': '-mute all', 'set_logging_handler': 'interactive', 'silent': True}


https://docs.anaconda.com/anaconda/install



INFO:pyrosetta.rosetta:Found rosetta database at: /home/pleung/.conda/envs/cereal/lib/python3.7/site-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.50+release.1295438cd4bd2be39c9dbbfab8db669ab62415ab 2020-12-12T00:30:01] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


# Now do nocys

In [ ]:
import logging
from pyrosetta.distributed.cluster.core import PyRosettaCluster
logging.basicConfig(level=logging.INFO)

pdbs = glob(os.path.join(os.getcwd(), "05_selected/nocys/good_trp/*.pdb"))

options = { 
    "-out:level": "300",
    "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
    "-indexed_structure_store:fragment_store": "/net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5",
    "-quasisymmetry": "true",
    "-detect_disulf": "false",
    "-dunbrack_prob_buried": "0.8",
    "-dunbrack_prob_nonburied": "0.8", 
    "-dunbrack_prob_buried_semi": "0.8", 
    "-dunbrack_prob_nonburied_semi": "0.8",
}

def create_tasks(pdbs, options):
    for pdb_path in pdbs:
        tasks = {"options": "-corrections::beta_nov16 true"}
        tasks["extra_options"] = options
        tasks["-s"] = pdb_path
        tasks["out_path"] = "06_states"
        tasks["set_logging_handler"] = "interactive"
        yield tasks
        
if not os.getenv("DEBUG"):
    output_path = os.path.join(os.getcwd(), "06_cleanup_nocys")
    PyRosettaCluster(
        tasks=create_tasks(pdbs, options),
        client=client,
        scratch_dir=output_path,
        output_path=output_path,
    ).distribute(protocols=[linkres_msd])

`conda env export --prefix /home/pleung/.conda/envs/cereal > environment.yml`
to reproduce this simulation later.
INFO:pyrosetta.distributed:maybe_init performing pyrosetta initialization: {'options': '-run:constant_seed 1 -multithreading:total_threads 1', 'extra_options': '-mute all', 'set_logging_handler': 'interactive', 'silent': True}
INFO:pyrosetta.rosetta:Found rosetta database at: /home/pleung/.conda/envs/cereal/lib/python3.7/site-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.50+release.1295438cd4bd2be39c9dbbfab8db669ab62415ab 2020-12-12T00:30:01] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


https://docs.anaconda.com/anaconda/install



# Look at scores
There is certainly a less embarrassing way to do this but at least this way is vectorized, so it should scale very well

In [ ]:
def read_scorefile(scores):
    import pandas as pd
    scores = pd.read_json(scores, orient="records", typ="frame", lines=True)
    scores = scores.T
    mat = scores.values
    n = mat.shape[0]
    dicts = list(mat[range(n), range(n)])
    index = scores.index
    tabulated_scores = pd.DataFrame(dicts, index=index)
    return tabulated_scores
    
output_path = os.path.join(os.getcwd(), "06_cleanup")
scores = os.path.join(output_path, "scores.json")
scores_df = read_scorefile(scores)
scores_df.head()

# Analyze sequence and residue metrics

GLY in middle loop GLY in interface, GLY positions, DSSP (compute glycine count on df)

TRP SASA X, Y, (compute delta on df)
TRP nearby aromatic non-TRP neighbors

HIS in interface SASA X, Y, (compute delta on df)  
HIS in interface



(compute sequence recovery of parent in df)


In [ ]:
sns.set(
    context="talk",
    font_scale=2, # make the font larger; default is pretty small
    style="ticks", # make the background white with black lines
    palette="colorblind" # a color palette that is colorblind friendly!
)

fig = plt.figure(figsize=(30,20), tight_layout=True)
sns.histplot(x="rmsd_final_X", y="rmsd_final_Y", data=scores_df, bins=20)
fig.savefig("figs/06_rmsd_final.png")


In [10]:
no_explode = scores_df[(scores_df.rmsd_final_X <= 0.6) & (scores_df.rmsd_final_Y <= 0.6)]
len(no_explode)

36

In [38]:
import pyrosetta
pyrosetta.distributed.init()
import pyrosetta.distributed.io as io
from pyrosetta.distributed import cluster

final_pdbs = glob(os.path.join(os.getcwd(), "06_states/*XY.pdb"))
to_inspect = []
with open("final_to_inspect.list", "w+") as f:
    for pdb in final_pdbs:
        basename_no_ext = pdb.split("_XY")[0]
        decoys = list(no_explode.index)
        for decoy in decoys:
            decoy_basename = cluster.get_scores_dict(decoy)["instance"]["tasks"]["-s"].split("/")[-1].replace(".pdb", "")
            if decoy_basename in basename_no_ext:
                decoys.remove(decoy)
                print(pdb.replace("X", "", 1), file=f)

In [49]:
os.makedirs("06_to_order", exist_ok=True)
tag = "GHHHHHHGS"
all_id = {}
with open("final_to_inspect.list", "r") as f:
    with open("06_to_order/all.fasta", "w+") as out:
        for i, line in enumerate(f):
            all_id[f"pf_{i}"] = f"{line.rstrip()}"
            print(f">pf_{i}", file=out)
            print(tag+io.pose_from_file(line.rstrip()).pose.sequence(), file=out)
all_id

{'pf_0': '/mnt/home/pleung/projects/bistable_bundle/r3/hinges/06_states/2021.03.08.22.24.53.141698_018487d85448435ca5a39dcf684aba6d_Y.pdb',
 'pf_1': '/mnt/home/pleung/projects/bistable_bundle/r3/hinges/06_states/2021.03.08.22.24.53.141698_5a28ccf4f05148479b48562ed2a419ab_Y.pdb',
 'pf_2': '/mnt/home/pleung/projects/bistable_bundle/r3/hinges/06_states/2021.03.08.22.24.53.141698_3e5d1e2bbaf4423c87baf7b831f63c68_Y.pdb',
 'pf_3': '/mnt/home/pleung/projects/bistable_bundle/r3/hinges/06_states/2021.03.08.22.24.53.141698_5d3c1ea5239f45d4b31f88bc284b1581_Y.pdb',
 'pf_4': '/mnt/home/pleung/projects/bistable_bundle/r3/hinges/06_states/2021.03.08.22.24.53.141698_8a78b774678047b7ab1e8409de147255_Y.pdb',
 'pf_5': '/mnt/home/pleung/projects/bistable_bundle/r3/hinges/06_states/2021.03.08.22.24.53.141698_5f14d50d11534d65bd713dca7bf0226d_Y.pdb',
 'pf_6': '/mnt/home/pleung/projects/bistable_bundle/r3/hinges/06_states/2021.03.08.22.24.53.141698_c77c043413c34f8bbcdab1089e970a8d_Y.pdb',
 'pf_7': '/mnt/home/

In [52]:
import json
with open("06_to_order/all_ids.json", "w+") as f:
    print(json.dumps(all_id), file=f)

In [ ]:
test = json.loads("06_to_order/all_ids.json")
test


In [1]:
from pyrosetta.distributed import cluster
with open("final_to_inspect.list", "r") as f:
    for line in f:
        cluster.get_scores_dict(line.rstrip()) #["instance"]["tasks"]["-s"].split("/")[-1].replace(".pdb", "")

https://docs.anaconda.com/anaconda/install



FileNotFoundError: [Errno 2] No such file or directory: 'final_to_inspect.list'

### Unused blocks

In [4]:
import pyrosetta
from pyrosetta.distributed import cluster
import pyrosetta.distributed.io as io

flags = """
-multithreading:total_threads 4
-quasisymmetry 1
-corrections::beta_nov16 true
-detect_disulf false
-holes:dalphaball /home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc
-indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5
-dunbrack_prob_buried 0.5
-dunbrack_prob_nonburied 0.5
-dunbrack_prob_buried_semi 0.5
-dunbrack_prob_nonburied_semi 0.5
"""
pyrosetta.distributed.init(" ".join(flags.replace("\n\t", " ").split()))
t = linkres_msd(
    None,
    **{
        "-s": "/mnt/home/pleung/projects/bistable_bundle/r4/hinges/05_msd_runs_2/1/73cd779ec93e2fd176eadc38444ef94803dc50a041b95971.pdb",
        "out_path": "test",
    }
)

core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] ################ Cloning pose and building neighbor graph ################
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] Ensure that pose is either scored or has update_residue_neighbors() called
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] before using NeighborhoodResidueSelector for maximum performance!
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] ##########################################################################
core.chemical.AtomICoor: {0} [ WARNING ] IcoorAtomID::atom_id(): Cannot get atom_id for POLYMER_LOWER of residue LEU 1.  Returning BOGUS ID instead.
core.conformation.Residue: {0} [ WARNING ] missing an atom: 1  H   that depends on a nonexistent polymer connection!
core.conformation.Residue: {0} [ WARNING ]  --> generating it using idealized coordinates.
core.chemical.AtomICoor: {0} [ WARNING ] IcoorAtomID::at

In [9]:
t.scores

{'X_nocys_path': '/mnt/home/pleung/projects/bistable_bundle/r4/hinges/test/73cd779ec93e2fd176eadc38444ef94803dc50a041b95971_X_nocys.pdb',
 'X_path': '/mnt/home/pleung/projects/bistable_bundle/r4/hinges/test/73cd779ec93e2fd176eadc38444ef94803dc50a041b95971_X.pdb',
 'X_seq': 'SSEKEELRERLVKHVVENAKRKGDDTEEAREAAREAFETVRETMERAGFSDSRMLRIAIQAIMAAVRLAVRKGDDISKMVRRVAKAVAEAARCAKEDGDTSSEEKRAAIQLVAQVARLAQHRGDDWEEVLRAAREAVCIVKEAARRAGITSPEEFKKAMEEIIKRVREAAQKGNDLSEAARQAAEEFRKKAEELKRRGDV',
 'Y_nocys_path': '/mnt/home/pleung/projects/bistable_bundle/r4/hinges/test/73cd779ec93e2fd176eadc38444ef94803dc50a041b95971_Y_nocys.pdb',
 'Y_path': '/mnt/home/pleung/projects/bistable_bundle/r4/hinges/test/73cd779ec93e2fd176eadc38444ef94803dc50a041b95971_Y.pdb',
 'Y_seq': 'SSEKEELRERLVKHVVENAKRKGDDTEEAREAAREAFETVRETMERAGFSDSRMLRIAIQAIMAAVRLAVRKGDDISKMVRRVAKAVAECARAAKEDGDTSSEEKRAAIQLVCQVARLAQHRGDDWEEVLRAAREAVEIVKEAARRAGITSPEEFKKAMEEIIKRVREAAQKGNDLSEAARQAAEEFRKKAEELKRRGDV',
 'closure_type_X': 'strict_remodel',
 'closu

In [3]:
inp_x = pyrosetta.rosetta.core.pose.Pose(
    t.pose, t.pose.chain_begin(1), t.pose.chain_end(1)
)

for key, value in t.pose.scores.items():
    pyrosetta.rosetta.core.pose.setPoseExtraScore(inp_x, key, value)

t2 = score(io.to_packed(inp_x))

inp_y = pyrosetta.rosetta.core.pose.Pose(
    t.pose, t.pose.chain_begin(2), t.pose.chain_end(2)
)
sw = pyrosetta.rosetta.protocols.simple_moves.SwitchChainOrderMover()
sw.chain_order("1")
sw.apply(inp_y)
for key, value in t.pose.scores.items():
    pyrosetta.rosetta.core.pose.setPoseExtraScore(inp_y, key, value)

t3 = score(io.to_packed(inp_y))

scores = t.pose.scores
updates = []
keys = ["HIS_burial", "TRP_burial", "buried_HIS", "near_int_GLY", "near_trp_ARO"]

for chain, score_dict in [("X", dict(t2.pose.scores)), ("Y", dict(t3.pose.scores))]:
    updates.append(
        {f"{key}_{chain}": value for key, value in score_dict.items() if key in keys}
    )
scores.update({**updates[0], **updates[1]})

NameError: name 't' is not defined